In [1]:
import pybamm

In [2]:
model = pybamm.lithium_ion.SPM(build=False)
print(model.submodels["thermal"])

In [3]:
class MyThermalModel (pybamm.models.submodels.thermal.BaseThermal):
    def __init__(self, param, options=None, x_average=False):
        super().__init__(param, options=options, x_average=x_average)

    def get_fundamental_variables(self):
        T_cylinder = pybamm.Variable("Cylinder temperature", domain = "full cell")
        T = pybamm.boundary_value(T_cylinder,'left')
        T_x_av = pybamm.PrimaryBroadcast(T, ["current collector"])
        T_dict = {
            "negative current collector": T_x_av,
            "positive current collector": T_x_av,
            "x-averaged cell": T_x_av,
            "volume-averaged cell": T,
        }
        for domain in ["negative electrode", "separator", "positive electrode"]:
            T_dict[domain] = pybamm.PrimaryBroadcast(T_x_av, domain)

        variables = self._get_standard_fundamental_variables(T_dict)
        #variables = []

        # define parameters
        R = pybamm.Parameter("Cell radius [m]")
        k = pybamm.Parameter("Heat conductivity [W/m2-K]")
        h = pybamm.Parameter("Convection heat transfer coefficient [W/m2-K]")
        T0 = pybamm.Parameter("Initial temperature [K]")
        Height = pybamm.Parameter("Cell height [m]")
        Tamb = pybamm.Parameter("Ambient temperature [K]")
        m = pybamm.Parameter("mass [kg]")
        Cp = pybamm.Parameter("Heat Capacity of the cell [J.K-1]")
        
        
        # update variables
        variables[R.name] = R
        variables [k.name] = k
        variables[h.name] = h
        variables[Cp.name] = Cp
        variables[T0.name] = T0
        variables[Height.name] = Height
        variables[Tamb.name] = Tamb
        variables[m.name] = m
        variables[T_cylinder.name] = T_cylinder
            
        return variables

            
    def get_coupled_variables(self,variables):
        variables.update(self._get_standard_coupled_variables(variables))

        #std_vars = self._get_standard_coupled_variables(variables)
        # Remove conflicting keys if they exist
        #for key in ["Throughput capacity [A.h]"]:
            #if key in std_vars:
                #del std_vars[key]
        #variables.update(std_vars)

        T = variables["Cylinder temperature"]
        #T=variables["Volume-averaged cell temperature [K]"]
        R = variables["Cell radius [m]"]
        k = variables["Heat conductivity [W/m2-K]"]
        h = variables["Convection heat transfer coefficient [W/m2-K]"]
        Height = variables["Cell height [m]"]
        Tamb = variables["Ambient temperature [K]"]
        m = variables["mass [kg]"]

        # derived parameters
        T_s = pybamm.surf(T)
        A = 2 * 3.14 * R * Height
        V = 3.14 * (R**2) * Height
        ro = m/V 
        Q_cool=h*(T-Tamb)

        variables.update({
            "Cooling [W]": Q_cool,
            "Area [m^2]": A,
            "Volume [m^3]": V,
            "density [kg/m^3]": ro
        })

        return variables

    def set_rhs(self,variables):

        T = variables["Cylinder temperature"]
        Cp = variables["Heat Capacity of the cell [J.K-1]"]
        ro = variables["density [kg/m^3]"]
        k = variables["Heat conductivity [W/m2-K]"]
        V = variables["Volume [m^3]"]
        Q_heat=variables["Total heating [W]"]

        dTdt = (1/(ro * Cp))*pybamm.div(k*pybamm.grad(T)) + Q_heat / (V * ro * Cp)
        self.rhs = {T: dTdt}

    def set_initial_conditions(self,variables):
        T0 = variables["Initial temperature [K]"]
        T=variables["Cylinder temperature"]
        self.initial_conditions={T: T0}  

    def  set_boundary_conditions(self,variables):
        T=variables["Cylinder temperature"]
        Q_cool=variables["Cooling [W]"]
        k = variables["Heat conductivity [W/m2-K]"]
        
        # boundary conditions
        lbc = pybamm.Scalar(0)
        rbc = - Q_cool / k
        self.boundary_conditions = {T: {"left": (lbc, "Neumann"), "right": (rbc, "Neumann")}}

    def output_variables(self,variables):   
        T=variables["Cylinder temperature"]
        T_s = pybamm.surf(T)
        Q_heat=variables["Total heating [W]"]
        
        # define output variables
        self.variables = {
            "Time [s]": pybamm.t,
            "Heat generation [W]": Q_heat,
            "Surface temperature [K]": T_s,
            "Cell Temperature [K]":T,
        }  

In [4]:
param1=model.default_parameter_values

param2 = pybamm.ParameterValues({"Heat conductivity [W/m2-K]": 0.2,
"Convection heat transfer coefficient [W/m2-K]": 40, 
"Initial temperature [K]": 300, "Cell height [m]": 0.07, 
"Cell radius [m]": 0.01, "Ambient temperature [K]": 300, 
"Heat Capacity of the cell": 1200, "mass [kg]":0.07})

submodel=MyThermalModel(model.param)
model.submodels["thermal"] = MyThermalModel(model.param)
model.build_model()


geometry1 = model.default_geometry
submesh_types1 = model.default_submesh_types
var_pts1 = model.default_var_pts

R = pybamm.Parameter("Cell radius [m]")
# define geometry
r_cell = pybamm.SpatialVariable(
    "r_cell", domain=["full cell"], coord_sys="cylindrical"
)
geometry2 = {
    "full cell": {r_cell: {"min": pybamm.Scalar(0), "max": R}}
}

geometry = {**geometry1, **geometry2}
# mesh and discretise
submesh_types2 = {"full cell": pybamm.Uniform1DSubMesh}
submesh_types = {**submesh_types1, **submesh_types2}
var_pts2 = {r_cell: 50}

var_pts = {**var_pts1,**var_pts2}

param1.update(param2,check_already_exists = False)
param = param1


param.process_geometry(geometry)

mesh=pybamm.Mesh(geometry, submesh_types, var_pts)
spatial_methods = {"full cell": pybamm.FiniteVolume()}
disc = pybamm.Discretisation(mesh, spatial_methods)

param.process_model(submodel)
disc.process_model(submodel)

ModelError: Cannot process parameters for empty model

In [ ]:
solver=pybamm.ScipySolver()
t_eval=np.linspace(0, 7200, 100)
solution= solver.solve(model, t_eval)

In [ ]:
solution.plot(["Surface temperature [K]", "Heat generation [W]", "Cell Temperature [K]"])

In [ ]:
pybamm.__version__

'25.1.1'